# Klasik NN ile belli bir değer aralığındaki bağımlı değişkenlerin kuadratik denkelemdeki çıktılarına göre model oluşturma
<img id="equationview" name="equationview" src="https://latex.codecogs.com/svg.latex?ax%5E%7B2%7D%20&amp;plus;%20bx%20&amp;plus;%20c%20%3D%200">

a ∈ [1,4],
b ∈ [-3,-1],
c ∈ [-4,-2],

<img src="https://latex.codecogs.com/svg.latex?\Large&space;x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}" title="\Large x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}" />

Yukarıdaki değer arlığındaki tam sayılar ile kuadratik denklemin sonuçları hesaplanacak ve 2 katmanlı bir yapay sinir ağında eğitilecek. Eğitim sonunda değer aralıklarında kalan ondalık değerler ile model teste tabi tutulacaktır. <br>
**Pytorch** framework'ü kullanılmıştır.

In [380]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math

#Define the model 
import torch
import torch.nn as nn
import torch.nn.functional as F

print(torch.__version__)

In [381]:
X = list()
Y = list()
for a in range(1,5):
    for b in range(-3,0):
        for c in range(-4,-1):
            X.append([a,b,c])
            y = (-b + math.sqrt(b*b - 4*a*c)) / 2*a
            Y.append(y)
            
print(len(X))
print(Y)

In [382]:
plt.plot(Y, marker = 'o')
plt.show()

In [383]:
x_train = torch.tensor(X, dtype=torch.float)
y_train = torch.tensor(Y, dtype=torch.float)

In [384]:
print(x_train.shape)
print(y_train.shape)

In [385]:
#Define training hyperprameters.
batch_size = 36
num_epochs = 500
learning_rate = 0.001
size_hidden = 10

#Calculate some other hyperparameters based on data.  
batch_no = len(x_train) // batch_size  #batches
inp_dim = x_train.shape[1] #Number of columns in input matrix
n_output = 1

In [386]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :",device)
class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

net = Net(inp_dim, size_hidden, n_output)

In [387]:
datasets = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(datasets)
print(datasets)

In [388]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [389]:
for epoch in range(num_epochs):
    for x, y in train_iter:
        output = net(x)
        l = loss(output, y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print("epoch {} loss: {:.4f}".format(epoch + 1, l.item()))

In [390]:
print(net(torch.tensor([1.2,-2.9,-3.5])))
print(y_train[30])